In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings("ignore")

# Set default plotly template
pio.templates.default = "plotly_white"

## 1. Load and Prepare Data

In [2]:
file_path = r'cleaned_data/fact_stores_with_info.csv'

print(f"Loading data from {file_path}...")
df = pd.read_csv(file_path)

df = df[df['mall_id'] == 22]

# Convert date
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
# Create Period column for aggregation
df['week_period'] = df['date'].dt.to_period('W')

print("Data loaded. Rows:", len(df))

Loading data from cleaned_data/fact_stores_with_info.csv...
Data loaded. Rows: 31184


In [3]:
# 1. Basic Cleaning
df = df.dropna(subset=['sales_eur', 'gla'])
df = df[(df['gla'] > 0)]

# Calculate basic store values
df['margin_eur'] = df['sales_eur'] - df['costs_eur']

# --- Absolute Metrics ---
df['sales_per_gla'] = df['sales_eur'] / df['gla']
df['people_in_per_gla'] = df['people_in'] / df['gla']
df['margin_per_gla'] = df['margin_eur'] / df['gla']
# Capture Rate (Absolute)
df['capture_rate'] = df.apply(lambda x: x['people_in'] / x['people_window_flow'] if x['people_window_flow'] > 0 else None, axis=1)


# --- Relative Metrics Calculation ---

# 2. Mall-Level Aggregation (Monthly)
print("Calculating Mall Aggregates...")
mall_stats = df.groupby(['mall_id', 'week_period']).agg(
    mall_total_sales=('sales_eur', 'sum'),
    mall_total_footfall=('people_in', 'sum'),
    mall_total_margin=('margin_eur', 'sum'),
    mall_total_gla=('gla', 'sum')
).reset_index()

# 3. Merge back
df = df.merge(mall_stats, on=['mall_id', 'week_period'], how='left')

# 4. Calculate Relative Indices
# Densities
mall_sales_density = df['mall_total_sales'] / df['mall_total_gla']
mall_footfall_density = df['mall_total_footfall'] / df['mall_total_gla']
mall_margin_density = df['mall_total_margin'] / df['mall_total_gla']

# Indices (Store Density / Mall Density)
df['relative_sales_index'] = df['sales_per_gla'] / mall_sales_density
df['relative_footfall_index'] = df['people_in_per_gla'] / mall_footfall_density
df['relative_margin_index'] = df['margin_per_gla'] / mall_margin_density

print("Metrics calculated. Sample rows:")
cols_to_show = ['store_code', 'mall_id', 'sales_per_gla', 'relative_sales_index']
display(df[cols_to_show].head())

Calculating Mall Aggregates...
Metrics calculated. Sample rows:


,store_code,mall_id,sales_per_gla,relative_sales_index
0,1092470,22,535.64635,0.778485
1,1088229,22,1672.68250,2.431005
2,1046340,22,1451.24577,2.109178
3,1095669,22,1131.10400,1.643898
4,1095953,22,2037.57188,2.961319


## 3. Descriptive Statistics

In [4]:
def generate_stats(group_col):
    unique_stores = df.drop_duplicates(subset=['store_code'])
    
    stats = unique_stores.groupby(group_col).agg(
        store_count=('store_code', 'count'),
        total_gla=('gla', 'sum'),
        avg_gla_per_store=('gla', 'mean')
    ).reset_index()
    
    return stats

print("### Stats by Category (bl1_label)")
display(generate_stats('bl1_label'))

print("### Stats by Block Type")
display(generate_stats('block_type'))

print("### Stats by GLA Category")
display(generate_stats('gla_category'))

### Stats by Category (bl1_label)


,bl1_label,store_count,total_gla,avg_gla_per_store
0,Bags & Footwear & Accessories,3,275.0,91.666667
1,Culture & Media & Technology,5,616.0,123.200000
2,Fashion apparel,31,21559.0,695.451613
3,Fitness,1,1974.0,1974.000000
4,Food & Beverage Services,25,3921.5,156.860000
5,Food Stores & Mass Merchandise,2,14630.0,7315.000000
6,Gifts,2,730.0,365.000000
7,Health & Beauty,17,3638.0,214.000000
8,Home,2,854.0,427.000000
9,Jewellery,6,861.0,143.500000


### Stats by Block Type


,block_type,store_count,total_gla,avg_gla_per_store
0,CELL,94,53175.0,565.691489
1,KIOSK,11,585.5,53.227273


### Stats by GLA Category


,gla_category,store_count,total_gla,avg_gla_per_store
0,LARGE UNITS,4,26699.0,6674.7500
1,MSU,13,13273.0,1021.0000
2,SMALL UNITS,88,13788.5,156.6875


## 4. Trend Analysis by Category (bl1_label)
Calculating the linear trend (slope) and significance (p-value) of performance metrics over time.

In [5]:
import numpy as np
import plotly.graph_objects as go

# Prepare time index for regression
df['time_index'] = df['date'].map(pd.Timestamp.toordinal)

def calculate_trend_stats(data, metric, time_col='time_index'):
    try:
        # Remove NaNs for the specific metric
        clean_data = data[[metric, time_col]].dropna() # Select only needed columns and copy
        if len(clean_data) < 10: # Need enough points
            return np.nan, np.nan, np.nan
        
        # Standardize metric (Z-score)
        val_mean = clean_data[metric].mean()
        val_std = clean_data[metric].std()
        
        if val_std == 0: # Avoid division by zero if constant
             return 0.0, 1.0, 0.0
             
        clean_data[metric] = (clean_data[metric] - val_mean) / val_std
        
        # Fit linear regression
        model = smf.ols(f"{metric} ~ {time_col}", data=clean_data).fit()
        return model.params[time_col], model.pvalues[time_col], model.tvalues[time_col]
    except Exception:
        return np.nan, np.nan, np.nan

metrics = [
    'sales_per_gla', 'margin_per_gla', 'people_in_per_gla', 'capture_rate',
    # 'relative_sales_index', 'relative_margin_index', 'relative_footfall_index'
    'store_average_dwell_time'
]

results_list = []

print("Calculating trends (slope, p-value, t-value)...")
labels = df['bl1_label'].unique()

for label in labels:
    if pd.isna(label): continue
    subset = df[df['bl1_label'] == label]
    for metric in metrics:
        slope, p_val, t_val = calculate_trend_stats(subset, metric)
        results_list.append({
            'bl1_label': label,
            'metric': metric,
            'slope': slope,
            'p_value': p_val,
            't_value': t_val
        })

stats_df = pd.DataFrame(results_list)

# Create matrices for plotting
t_matrix = stats_df.pivot(index='bl1_label', columns='metric', values='t_value')
slope_matrix = stats_df.pivot(index='bl1_label', columns='metric', values='slope')
p_matrix = stats_df.pivot(index='bl1_label', columns='metric', values='p_value')

# Create text matrix (Slope + Stars)
def fmt_cell(slope, p):
    if pd.isna(slope): return ""
    stars = ""
    if p < 0.01: stars = "***"
    elif p < 0.05: stars = "**"
    elif p < 0.1: stars = "*"
    
    # Format depending on magnitude
    if abs(slope) < 0.0001:
        return f"{slope:.2e}{stars}"
    else:
        return f"{slope:.4f}{stars}"

text_matrix = slope_matrix.copy()
for col in text_matrix.columns:
    text_matrix[col] = [fmt_cell(s, p) for s, p in zip(slope_matrix[col], p_matrix[col])]

# Plot Heatmap
fig = go.Figure(data=go.Heatmap(
    z=slope_matrix,
    x=t_matrix.columns,
    y=t_matrix.index,
    text=text_matrix,
    texttemplate="%{text}",
    colorscale='RdYlGn',
    zmid=0,
    colorbar=dict(title="Trend Strength (slope)")
))

fig.update_layout(
    title="Performance Trends by Category (Slope & Significance*)",
    height=800,
    xaxis_title="Metric",
    yaxis_title="Category",
    template="plotly_white"
)

fig.show()
print("* p<0.1, ** p<0.05, *** p<0.01. Color indicates t-value (direction and strength).")

Calculating trends (slope, p-value, t-value)...


* p<0.1, ** p<0.05, *** p<0.01. Color indicates t-value (direction and strength).


In [6]:
results_list = []

print("Calculating trends (slope, p-value, t-value)...")
labels = df['bl2_label'].unique()

for label in labels:
    if pd.isna(label): continue
    subset = df[df['bl2_label'] == label]
    for metric in metrics:
        slope, p_val, t_val = calculate_trend_stats(subset, metric)
        results_list.append({
            'bl2_label': label,
            'metric': metric,
            'slope': slope,
            'p_value': p_val,
            't_value': t_val
        })

stats_df = pd.DataFrame(results_list)

# Create matrices for plotting
t_matrix = stats_df.pivot(index='bl2_label', columns='metric', values='t_value')
slope_matrix = stats_df.pivot(index='bl2_label', columns='metric', values='slope')
p_matrix = stats_df.pivot(index='bl2_label', columns='metric', values='p_value')

# Create text matrix (Slope + Stars)
def fmt_cell(slope, p):
    if pd.isna(slope): return ""
    stars = ""
    if p < 0.01: stars = "***"
    elif p < 0.05: stars = "**"
    elif p < 0.1: stars = "*"
    
    # Format depending on magnitude
    if abs(slope) < 0.0001:
        return f"{slope:.2e}{stars}"
    else:
        return f"{slope:.4f}{stars}"

text_matrix = slope_matrix.copy()
for col in text_matrix.columns:
    text_matrix[col] = [fmt_cell(s, p) for s, p in zip(slope_matrix[col], p_matrix[col])]

# Plot Heatmap
fig = go.Figure(data=go.Heatmap(
    z=slope_matrix,
    x=t_matrix.columns,
    y=t_matrix.index,
    text=text_matrix,
    texttemplate="%{text}",
    colorscale='RdYlGn',
    zmid=0,
    colorbar=dict(title="Trend Strength (slope)")
))

fig.update_layout(
    title="Performance Trends by Category (Slope & Significance*)",
    height=800,
    xaxis_title="Metric",
    yaxis_title="Category",
    template="plotly_white"
)

fig.show()
print("* p<0.1, ** p<0.05, *** p<0.01. Color indicates t-value (direction and strength).")

Calculating trends (slope, p-value, t-value)...


* p<0.1, ** p<0.05, *** p<0.01. Color indicates t-value (direction and strength).


In [7]:
results_list = []

print("Calculating trends (slope, p-value, t-value)...")
labels = df['bl3_label'].unique()

for label in labels:
    if pd.isna(label): continue
    subset = df[df['bl3_label'] == label]
    for metric in metrics:
        slope, p_val, t_val = calculate_trend_stats(subset, metric)
        results_list.append({
            'bl3_label': label,
            'metric': metric,
            'slope': slope,
            'p_value': p_val,
            't_value': t_val
        })

stats_df = pd.DataFrame(results_list)

# Create matrices for plotting
t_matrix = stats_df.pivot(index='bl3_label', columns='metric', values='t_value')
slope_matrix = stats_df.pivot(index='bl3_label', columns='metric', values='slope')
p_matrix = stats_df.pivot(index='bl3_label', columns='metric', values='p_value')

# Create text matrix (Slope + Stars)
def fmt_cell(slope, p):
    if pd.isna(slope): return ""
    stars = ""
    if p < 0.01: stars = "***"
    elif p < 0.05: stars = "**"
    elif p < 0.1: stars = "*"
    
    # Format depending on magnitude
    if abs(slope) < 0.0001:
        return f"{slope:.2e}{stars}"
    else:
        return f"{slope:.4f}{stars}"

text_matrix = slope_matrix.copy()
for col in text_matrix.columns:
    text_matrix[col] = [fmt_cell(s, p) for s, p in zip(slope_matrix[col], p_matrix[col])]

# Plot Heatmap
fig = go.Figure(data=go.Heatmap(
    z=slope_matrix,
    x=t_matrix.columns,
    y=t_matrix.index,
    text=text_matrix,
    texttemplate="%{text}",
    colorscale='RdYlGn',
    zmid=0,
    colorbar=dict(title="Trend Strength (slope)")
))

fig.update_layout(
    title="Performance Trends by Category (Slope & Significance*)",
    height=800,
    xaxis_title="Metric",
    yaxis_title="Category",
    template="plotly_white"
)

fig.show()
print("* p<0.1, ** p<0.05, *** p<0.01. Color indicates t-value (direction and strength).")

Calculating trends (slope, p-value, t-value)...


* p<0.1, ** p<0.05, *** p<0.01. Color indicates t-value (direction and strength).
